### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

Source file: C:\Users\ping\Downloads\df_finviz_2025-06-16_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-06-16_df_finviz_stocks_etfs.parquet
Processing for date: 2025-06-16


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-06-13_stocks_etfs.parquet ---
Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: object(111)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS this Y,EPS next Y,EPS past 5Y,EPS next 5Y,Sales past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,161,VONV,Vanguard Russell 1000 Value Index ETF,-,Financial,Exchange Traded Fund,USA,NASD,-,-,-,-,-,-,-,-,-,-,1.98%,1.65,3/25/2025,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.42,0.33M,-,-,-,-,-,-,-,-,-,-,-0.45%,1.35%,2.03%,-2.08%,8.61%,2.57%,0.89,0.92,0.63%,0.76%,0.18%,3.29%,0.55%,-1.13%,16.26%,-5.26%,16.26%,71.68 - 87.97,-5.26%,254.41%,54.83,-,9/22/2010,Yes,Yes,-,-0.31%,-0.68%,-,786.92K,0.64,"505,981",-,84.17,83.60,84.03,83.14,83.34,-0.99%,US Equities - US Style,Equities (Stocks),0.07%,873,12.20B,-68.78M,-0.56%,692.76M,6.02%,1.99B,19.46%,10.71%,8.24%,11.63%,"U.S., equity, value, Russell-1000, large-cap"
1,162,VFH,Vanguard Financials ETF,-,Financial,Exchange Traded Fund,USA,NYSE,-,-,-,-,-,-,-,-,-,-,1.80%,2.18,3/25/2025,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.18,0.10M,-,-,-,-,-,-,-,-,-,-,-2.59%,-1.79%,3.56%,-1.01%,22.28%,2.46%,1.02,1.73,1.06%,0.96%,-1.60%,2.32%,2.30%,-3.25%,19.93%,-5.25%,23.65%,97.83 - 127.67,-5.25%,825.55%,46.09,-,1/30/2004,Yes,Yes,-,-0.78%,-1.33%,-,532.45K,0.90,"480,540",-,123.56,121.92,122.16,120.67,120.97,-2.10%,US Equities - Industry Sector,Equities (Stocks),0.09%,402,12.17B,106.74M,0.88%,-140.54M,-1.14%,211.67M,1.77%,25.37%,14.93%,15.82%,"U.S., equity, financial"
2,163,SPEM,SPDR Portfolio Emerging Markets ETF,-,Financial,Exchange Traded Fund,USA,NYSE,-,-,-,-,-,-,-,-,-,-,2.54%,1.07,12/23/2024,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.64,4.09M,-,-,-,-,-,-,-,-,-,-,0.10%,0.65%,4.82%,4.35%,11.97%,9.49%,0.56,0.40,0.96%,0.63%,0.64%,4.97%,5.80%,-1.86%,22.19%,-2.03%,22.19%,34.38 - 42.88,-11.67%,188.12%,58.77,-,3/23/2007,Yes,Yes,-,0.02%,-1.55%,-,2.50M,0.59,"1,477,296",-,42.66,42.00,42.19,41.93,42.01,-1.52%,Global or ExUS Equities - Broad / Regional,Equities (Stocks),0.07%,3119,12.12B,468.95M,4.02%,869.21M,7.73%,1.25B,11.49%,14.75%,7.63%,7.11%,"Emerging, equity"


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [3]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")
    
    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace('-', '')
    df['Info'] = df[INFO_COLS].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[['Ticker', 'Info', 'MktCap AUM']].head(3))


--- Step 2: Engineering new features ---
Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,VONV,"Financial, Exchange Traded Fund, US Equities -...",12.20B
1,VFH,"Financial, Exchange Traded Fund, US Equities -...",12.17B
2,SPEM,"Financial, Exchange Traded Fund, Global or ExU...",12.12B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [4]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {'T': 1_000_000, 'B': 1_000, 'M': 1, 'K': 0.001}
    suffix = value_str[-1]
    
    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan

if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"
    
    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,NaN,12200.0,NaN,NaN,NaN,NaN,0.33,0.78692,-68.78,692.76,1990.00,12200.0
1,NaN,12170.0,NaN,NaN,NaN,NaN,0.10,0.53245,106.74,-140.54,211.67,12170.0
2,NaN,12120.0,NaN,NaN,NaN,NaN,4.09,2.50000,468.95,869.21,1250.00,12120.0


#### Part B: Convert Percentage Columns to Numeric

In [5]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col for col in df.columns if df[col].dtype == 'object' and df[col].str.endswith('%', na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace('%', ''), errors='coerce')
            new_name = f"{col} %" if '%' not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---
Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS this Y' to numeric and renamed to 'EPS this Y %'
  - Converted 'EPS next Y' to numeric and renamed to 'EPS next Y %'
  - Converted 'EPS past 5Y' to numeric and renamed to 'EPS past 5Y %'
  - Converted 'EPS next 5Y' to numeric and renamed to 'EPS next 5Y %'
  - Converted 'Sales past 5Y' to numeric and renamed to 'Sales past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise 

#### Part C: Convert Other String-Based Numeric Columns

In [6]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce')
            converted_count += 1
            
    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 32 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [7]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")
    
    # 1. Sort by the primary metric in descending order
    df.sort_values(by='MktCap AUM, M', ascending=False, inplace=True, na_position='last')
    print("Sorted DataFrame by 'MktCap AUM, M'.")
    
    # 2. Add a 'Rank' column based on the new sort order
    df['Rank'] = range(1, len(df) + 1)
    print("Added 'Rank' column.")
    
    # 3. Set 'Ticker' as the index
    if 'Ticker' in df.columns:
        df.set_index('Ticker', inplace=True)
        print("Set 'Ticker' as the index.")
    
    print("\nFinal DataFrame structure:")
    display(df[['Rank', 'Info', 'MktCap AUM, M']].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


C:\Users\ping\AppData\Local\Temp\ipykernel_14552\3337454481.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Rank'] = range(1, len(df) + 1)


,Rank,Info,"MktCap AUM, M"
Ticker,,,
MSFT,1,"Technology, Software - Infrastructure",3530160.0
NVDA,2,"Technology, Semiconductors",3464070.0
AAPL,3,"Technology, Consumer Electronics",2934140.0
AMZN,4,"Consumer Cyclical, Internet Retail",2251730.0
GOOGL,5,"Communication Services, Internet Content & Inf...",2126510.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [8]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file
        df.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine='pyarrow')
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: ..\data\2025-06-13_df_finviz_stocks_etfs.parquet

Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS this Y %,EPS next Y %,EPS past 5Y %,EPS next 5Y %,Sales past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MSFT,1,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3530160.0,36.70,31.35,2.52,13.07,10.97,44.34,50.89,43.30,10.71,0.69,2.41,8/21/2025,25.42,12.94,3.37,13.50,13.12,18.45,14.54,14.33,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.00,7320.00,98.51,1.47,-0.12,73.62,0.40,0.79,2.53,58.18,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,0.97,4.86,22.24,7.13,9.77,12.68,1.03,6.98,0.94,0.83,2.55,11.56,13.20,-1.14,37.75,-1.14,37.75,344.79 - 480.42,-1.14,595994.50,71.47,Apr 30/a,3/13/1986,Yes,Yes,228000.0,-0.35,-0.46,1.30,23.01000,0.73,16794463,515.98,478.87,476.65,479.18,472.76,474.96,-0.82,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3530160.0,1
NVDA,2,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,3464070.0,45.73,24.81,1.53,23.32,41.30,64.52,48.07,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,45.30,31.72,91.83,29.90,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.00,23400.00,95.96,4.08,-0.37,66.38,0.20,0.98,0.88,229.34,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,0.18,4.90,16.68,5.11,17.42,5.72,2.12,4.25,1.65,1.73,2.71,16.63,11.24,-2.09,63.90,-7.29,63.90,86.62 - 153.13,-7.29,425809.98,62.71,May 28/a,1/22/1999,Yes,Yes,36000.0,-0.34,-1.75,1.38,259.30000,0.68,177084632,171.67,145.00,142.46,143.58,140.85,141.97,-2.09,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",3464070.0,2
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,2934140.0,30.66,25.24,3.82,7.33,43.94,60.50,29.79,4.47,3.25,0.53,1.01,5/12/2025,16.11,6.41,1.42,6.27,8.51,15.41,8.03,8.51,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.00,14920.00,99.88,0.10,-1.92,63.82,-0.54,0.64,1.55,94.83,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,-3.66,-7.48,-7.98,-20.71,-5.17,-21.55,1.21,5.04,1.23,1.32,-2.70,-2.47,-12.48,-12.76,16.10,-24.47,16.10,169.21 - 260.10,-24.47,308705.92,40.72,May 01/a,12/12/1980,Yes,Yes,164000.0,-1.50,0.12,2.08,61.37000,0.84,51282817,228.26,199.20,199.44,200.37,195.70,196.45,-1.38,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",2934140.0,3
AMZN,4,Amazon.com Inc,"DJIA, NDX, S&P 500",Consumer Cyclical,Internet Retail,USA,NASD,2251730.0,34.59,29.23,2.01,3.46,7.36,22.92,108.20,28.82,9.25,NaN,NaN,-,0.00,6.13,1.32,11.93,17.23,36.89,17.21,17.86,8.62,62.33,71.88,10.08,650310.0,65940.0,16.38,0.33,10610.00,9490.00,89.45,10.58,-0.02,64.44,0.06,0.65,1.26,61.36,11.23,25.24,15.02,1.05,0.84,0.44,0.49,49.16,11.15,10.14,-0.69,0.88,7.15,-5.75,13.28,-3.32,1.34,5.08,1.04,1.20,2.05,8.94,4.41,-2.88,31.43,-12.54,39.90,151.61 - 242.52,-12.54,323100.02,59.90,